In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  |     Proc size: 118.9 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [ ]:
cd /content/drive/My\ Drive/Colab_Notebooks/BCI/p3net/

/content/drive/My Drive/Colab_Notebooks/BCI/p3net


In [ ]:
!cat 'models.ipynb'
!cat 'dataloader.ipynb'
!cat 'train_utils.ipynb'

import sys
sys.path.append('/content/drive/My\ Drive/Colab_Notebooks/BCI/p3net/')

{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"name":"models.ipynb","provenance":[],"collapsed_sections":[],"authorship_tag":"ABX9TyP7O0gb7Y0zO6D30y/LxnUk"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","metadata":{"id":"X2_mtlqBabm-"},"source":["# from google.colab import drive\n","# drive.mount('/content/drive/', force_remount=True)"],"execution_count":null,"outputs":[]},{"cell_type":"code","metadata":{"id":"D2hX0j-yacF_"},"source":["import torch\n","from torch.autograd import Variable\n","import torch.nn as nn\n","import numpy as np\n","import pdb\n","\n","# %%\n","def compute_conv_dim(dim_size, kernel_size, padding, stride):\n","    # compute the output dimensions of a convolutional layer\n","    out_dim = int((dim_size - kernel_size + 2 * padding) / stride + 1)\n","    return out_dim\n","\n","# %%\n","class CNN2D(torch.nn.Module):\n","\n","    def __init__(self,\n","                 input_size,  # 

In [ ]:
import torch 
import pandas as pd 
import pickle 

# import dataloader
# import train_utils
# import models

from dataloader import getTorch, EEGDataLoader
from train_utils import train_model
from models import CNN2DEncoder, CNNLSTM

In [ ]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    print("CUDA available!")
    dev = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')


get_data = getTorch.get_data 

# Load ERP data
dname = dict(nu   = 'data_NUsubjects.pickle')

#%% Hyperparameter settings 

num_epochs = 30
batch_size = 8
verbose = 2
learning_rate = 1e-3
weight_decay = 1e-4                   

CUDA available!


In [ ]:
!pip3 install mne

In [ ]:
import mne
import gc

for itemname, filename in dname.items():
    print('working with', filename)
    iname = itemname + '__'   

    if filename =='data_NUsubjects.pickle':
        d  =  {'conv_channels': [1, 256, 128, 64, 32, 16, 8], # convolutional layers and number of filters in each layer 
               'kernel_size':   [3, 3, 3, 3, 3, 3], # kernel size 
               'num_layers':    1, # number of LSTM layers
               'hidden_size':   128} # number of neurons in each LSTM layer 
  
    
    # EEG data loader 
    dd = EEGDataLoader(filename)
    
    # load subject specific data with subject data indicies  
    s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
    data = dd.subject_specific(s)
    print(data[0].keys())
        
    #% identify input size (channel x timepoints)
    timelength = data[0]['xtrain'].shape[2]
    chans      = data[0]['xtrain'].shape[1]
 
    datum = {}
    # get torch data loaders for each subject data 
    for ii in range(len(data)):
      datum[ii] = get_data(data[ii], batch_size, image = True, lstm = False, raw = False)
   
    # used for storing results later 
    results, models = {}, {}  
    table = pd.DataFrame(columns = ['Train_Loss', 'Val_Loss', 'Train_Acc', 
                                        'Val_Acc', 'Test_Acc', 'Epoch'])
    
    # for each subject in a given data type perform model selection
    for subjectIndex in datum:    
        dset_loaders = datum[subjectIndex]['dset_loaders']
        dset_sizes   = datum[subjectIndex]['dset_sizes']      
        
        input_size = datum[subjectIndex]['test_data']['x_test'].shape

        # define the encoder model                  
        encoder = CNN2DEncoder(kernel_size = d['kernel_size'], 
                               conv_channels = d['conv_channels'])      
        # we need to pass a sample data to get the output dimensionality of CNN 
        with torch.no_grad():
            x = torch.randn(input_size)
            outdim = encoder(x)
            batch_size, chans, H, W = outdim.size()
            
        # define the encoder-decoder model 
        model = CNNLSTM(input_size = H*W, 
                        cnn = encoder,
                        hidden_size = d['hidden_size'], 
                        num_layers  = d['num_layers'], 
                        batch_size  = 64,
                        dropout     = 0.2)      
        
        # define the optimizer and the loss function 
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
        criterion = torch.nn.CrossEntropyLoss()
        
        # move to GPU/CPU
        model.to(dev)  
        criterion.to(dev)           
        
        # the training loop
        best_model, train_losses, val_losses, train_accs, val_accs, info = train_model(model, dset_loaders, 
                                                                                       dset_sizes, criterion, 
                                                                                       optimizer, dev, 
                                                                                       lr_scheduler = None, 
                                                                                       num_epochs = num_epochs, 
                                                                                       verbose = verbose)    
         
        x_test = datum[subjectIndex]['test_data']['x_test'] 
        y_test = datum[subjectIndex]['test_data']['y_test'] 
        
        h=best_model.init_hidden(x_test.shape[0])
        preds = best_model(x_test.to(dev),h).detach()    
        preds_class = preds.data.max(1)[1]
        
        # accuracy metric 
        corrects = torch.sum(preds_class == y_test.data.to(dev))     
        test_acc = corrects.cpu().numpy()/x_test.shape[0]    
        print("Test Accuracy :", test_acc)    
        
        # save results       
        tab = dict(Train_Loss = train_losses[info['best_epoch']], 
                   Val_Loss   = val_losses[info['best_epoch']],
                   Train_Acc  = train_accs[info['best_epoch']],
                   Val_Acc    = val_accs[info['best_epoch']],   
                   Test_Acc   = test_acc, 
                   Epoch      = info['best_epoch'] + 1) 
        
        subjectIndexx = subjectIndex + 1
        table.loc[subjectIndexx] = tab
        
        results[subjectIndexx] = dict(train_losses = train_losses, val_losses = val_losses,
                                    train_accs = train_accs,     val_accs =  val_accs,                                
                                    ytrain = info['ytrain'],     yval= info['yval'])      
          
        # save models   
        fname = 'model/' + iname + 'S'+ str(subjectIndexx) + '_CNNLSTM_model_'
        torch.save(best_model.state_dict(), fname)
        torch.cuda.empty_cache()
         
        print('::: saving subject {} ::: \n {}'.format(subjectIndexx, table))         
        result_lstm_subspe = dict(table = table, results = results)
        torch.cuda.empty_cache()
        torch.cuda.memory_summary(device=None, abbreviated=False)         
          
    fname = "model/" + iname + '__S'+str(subjectIndexx)  + '_CNNLSTM_subspe_results'
    
    with open(fname, 'wb') as fp:
        pickle.dump(result_lstm_subspe, fp)  

working with data_NUsubjects.pickle
Loading subjects: 0
Loading subjects: 1
Loading subjects: 2
Loading subjects: 3
Loading subjects: 4
Loading subjects: 5
Loading subjects: 6
Loading subjects: 7
Loading subjects: 8
Loading subjects: 9
Loading subjects: 10
dict_keys(['xtrain', 'xvalid', 'xtest', 'ytrain', 'yvalid', 'ytest'])
Input data shape torch.Size([4732, 1, 16, 76])
Input data shape torch.Size([4629, 1, 16, 76])
Input data shape torch.Size([2435, 1, 16, 76])
Input data shape torch.Size([7727, 1, 16, 76])
Input data shape torch.Size([4026, 1, 16, 76])
Input data shape torch.Size([4374, 1, 16, 76])
Input data shape torch.Size([1302, 1, 16, 76])
Input data shape torch.Size([1393, 1, 16, 76])
Input data shape torch.Size([4910, 1, 16, 76])
Input data shape torch.Size([4449, 1, 16, 76])
Input data shape torch.Size([4458, 1, 16, 76])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1/30
train loss: 0.0826, acc: 0.5978
val loss: 0.0886, acc: 0.5562
Epoch 2/30
train loss: 0.0711, acc: 0.7113
val loss: 0.1220, acc: 0.4691
Epoch 3/30
train loss: 0.0515, acc: 0.8180
val loss: 0.1832, acc: 0.5258
Epoch 4/30
train loss: 0.0325, acc: 0.8975
val loss: 0.2297, acc: 0.5004
Epoch 5/30
train loss: 0.0218, acc: 0.9341
val loss: 0.3336, acc: 0.4877
Epoch 6/30
train loss: 0.0141, acc: 0.9577
val loss: 0.2465, acc: 0.5545
Epoch 7/30
train loss: 0.0113, acc: 0.9664
val loss: 0.3325, acc: 0.5199
Epoch 8/30
train loss: 0.0092, acc: 0.9753
val loss: 0.2865, acc: 0.5063
Epoch 9/30
train loss: 0.0080, acc: 0.9784
val loss: 0.4221, acc: 0.4844
Epoch 10/30
train loss: 0.0063, acc: 0.9829
val loss: 0.3878, acc: 0.4784
Epoch 11/30
train loss: 0.0071, acc: 0.9808
val loss: 0.3953, acc: 0.4691
Epoch 12/30
train loss: 0.0064, acc: 0.9808
val loss: 0.3109, acc: 0.5072
Epoch 13/30
train loss: 0.0055, acc: 0.9842
val loss: 0.4306, acc: 0.4632
Epoch 14/30
train loss: 0.0039, acc: 0.9894
val

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1/30
train loss: 0.0725, acc: 0.6982
val loss: 0.0828, acc: 0.6555
Epoch 2/30
train loss: 0.0431, acc: 0.8570
val loss: 0.1219, acc: 0.7046
Epoch 3/30
train loss: 0.0209, acc: 0.9374
val loss: 0.1525, acc: 0.6796
Epoch 4/30
train loss: 0.0133, acc: 0.9592
val loss: 0.2830, acc: 0.6486
Epoch 5/30
train loss: 0.0092, acc: 0.9732
val loss: 0.2901, acc: 0.6400
Epoch 6/30
train loss: 0.0078, acc: 0.9806
val loss: 0.1725, acc: 0.6529
Epoch 7/30
train loss: 0.0068, acc: 0.9803
val loss: 0.2053, acc: 0.6279
Epoch 8/30
train loss: 0.0045, acc: 0.9890
val loss: 0.2323, acc: 0.6649
Epoch 9/30
train loss: 0.0053, acc: 0.9847
val loss: 0.2312, acc: 0.6417
Epoch 10/30
train loss: 0.0040, acc: 0.9886
val loss: 0.2713, acc: 0.6270
Epoch 11/30
train loss: 0.0052, acc: 0.9886
val loss: 0.2615, acc: 0.6494
Epoch 12/30
train loss: 0.0052, acc: 0.9870
val loss: 0.2057, acc: 0.6038
Epoch 13/30
train loss: 0.0044, acc: 0.9875
val loss: 0.2801, acc: 0.6365
Epoch 14/30
train loss: 0.0037, acc: 0.9901
val

...

train loss: 0.0763, acc: 0.6585
val loss: 0.0733, acc: 0.7113
Epoch 2/30
train loss: 0.0522, acc: 0.8240
val loss: 0.1035, acc: 0.6634
Epoch 3/30
train loss: 0.0312, acc: 0.8992
val loss: 0.1030, acc: 0.6221
Epoch 4/30
train loss: 0.0185, acc: 0.9485
val loss: 0.1881, acc: 0.6139
Epoch 5/30
train loss: 0.0134, acc: 0.9613
val loss: 0.2583, acc: 0.6010
Epoch 6/30
train loss: 0.0082, acc: 0.9780
val loss: 0.2725, acc: 0.6188
Epoch 7/30
train loss: 0.0079, acc: 0.9778
val loss: 0.2687, acc: 0.5937
Epoch 8/30
train loss: 0.0075, acc: 0.9807
val loss: 0.2719, acc: 0.5856
Epoch 9/30
train loss: 0.0058, acc: 0.9847
val loss: 0.3008, acc: 0.5766
Epoch 10/30
train loss: 0.0055, acc: 0.9876
val loss: 0.2770, acc: 0.5523
Epoch 11/30
train loss: 0.0061, acc: 0.9839
val loss: 0.2198, acc: 0.6050
Epoch 12/30
train loss: 0.0033, acc: 0.9919
val loss: 0.4005, acc: 0.5612
Epoch 13/30
train loss: 0.0050, acc: 0.9878
val loss: 0.3073, acc: 0.5580
Epoch 14/30
train loss: 0.0052, acc: 0.9853
val loss: 0.32

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


train loss: 0.0798, acc: 0.6437
val loss: 0.0951, acc: 0.5633
Epoch 2/30
train loss: 0.0567, acc: 0.7923
val loss: 0.1013, acc: 0.5894
Epoch 3/30
train loss: 0.0317, acc: 0.8962
val loss: 0.1466, acc: 0.5597
Epoch 4/30
train loss: 0.0156, acc: 0.9532
val loss: 0.2264, acc: 0.5615
Epoch 5/30
train loss: 0.0102, acc: 0.9721
val loss: 0.2673, acc: 0.5624
Epoch 6/30
train loss: 0.0068, acc: 0.9811
val loss: 0.2420, acc: 0.5696
Epoch 7/30
train loss: 0.0075, acc: 0.9782
val loss: 0.2671, acc: 0.5526
Epoch 8/30
train loss: 0.0051, acc: 0.9854
val loss: 0.2677, acc: 0.5265
Epoch 9/30
train loss: 0.0073, acc: 0.9820
val loss: 0.3619, acc: 0.5400
Epoch 10/30
train loss: 0.0052, acc: 0.9852
val loss: 0.3563, acc: 0.5454
Epoch 11/30
train loss: 0.0032, acc: 0.9912
val loss: 0.4648, acc: 0.5328
Epoch 12/30
train loss: 0.0051, acc: 0.9858
val loss: 0.3592, acc: 0.5391
Epoch 13/30
train loss: 0.0033, acc: 0.9910
val loss: 0.3965, acc: 0.5418
Epoch 14/30
train loss: 0.0038, acc: 0.9890
val loss: 0.48

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


train loss: 0.0803, acc: 0.6234
val loss: 0.0830, acc: 0.6170
Epoch 2/30
train loss: 0.0566, acc: 0.7952
val loss: 0.1044, acc: 0.6242
Epoch 3/30
train loss: 0.0312, acc: 0.9013
val loss: 0.1547, acc: 0.6260
Epoch 4/30
train loss: 0.0162, acc: 0.9507
val loss: 0.2067, acc: 0.6493
Epoch 5/30
train loss: 0.0104, acc: 0.9699
val loss: 0.2756, acc: 0.5839
Epoch 6/30
train loss: 0.0080, acc: 0.9769
val loss: 0.2886, acc: 0.6152
Epoch 7/30
train loss: 0.0086, acc: 0.9762
val loss: 0.3336, acc: 0.5785
Epoch 8/30
train loss: 0.0047, acc: 0.9865
val loss: 0.2845, acc: 0.6108
Epoch 9/30
train loss: 0.0050, acc: 0.9863
val loss: 0.2873, acc: 0.5848
Epoch 10/30
train loss: 0.0057, acc: 0.9850
val loss: 0.3391, acc: 0.5937
Epoch 11/30
train loss: 0.0049, acc: 0.9865
val loss: 0.2753, acc: 0.6072
Epoch 12/30
train loss: 0.0037, acc: 0.9899
val loss: 0.4402, acc: 0.6063
Epoch 13/30
train loss: 0.0041, acc: 0.9881
val loss: 0.2869, acc: 0.6108
Epoch 14/30
train loss: 0.0042, acc: 0.9879
val loss: 0.44